<p style="font-size:32px; font-weight: bolder; text-align: center"> Rotations, equivariance and <br/> symmetry-adapted regression </p>
<p style="text-align: center"><i> authored by: <a href="mailto:michele.ceriotti@gmail.com"> Michele Ceriotti </a></i></p>

This notebook discusses the concept of equivariance, with a specific focus on the rotation group. We will learn about Cartesian rotations, spherical harmonics, Wigner matrices.
We will see how these concepts apply to some of the equivariant descriptors used in machine learning, and how it is possible to build simple regression models that yield rotationally equivariant predictions for vectorial or tensorial properties.

### Packages and dependencies
This module uses some utility functions from `scipy` and `spherical` to handle rotations, and `rascaline` to compute descriptors. 

In [ ]:
%matplotlib widget
# scwidgets import
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d
import chemiscope

import ipywidgets
from ipywidgets import FloatSlider, IntSlider, Checkbox, Dropdown, HBox, Layout, HTML, Text

from markdown import markdown as mdwn

import scwidgets
from scwidgets.check import (
    Check,
    CheckRegistry,
    assert_numpy_allclose,
    assert_numpy_floating_sub_dtype,
    assert_shape,
    assert_type,
)
from scwidgets.code import ParameterPanel, CodeInput
from scwidgets.cue import CueObject, CueFigure
from scwidgets.exercise import CodeExercise, TextExercise, ExerciseRegistry

In [ ]:
import numpy as np
import ase, ase.io
import itertools
from copy import deepcopy
from tqdm.notebook import tqdm

import rascaline
from metatensor import mean_over_samples, Labels, TensorMap, TensorBlock, slice_block

from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV

import scipy

from sphericart import SphericalHarmonics
sph = SphericalHarmonics(l_max=8) # initializes sph calculator

In [ ]:
from rotutils import *

In [ ]:
# set CSS style for code-hide
scwidgets.get_css_style()

### Answer settings

In [ ]:
exercise_registry = ExerciseRegistry(filename_prefix="module_03")
exercise_registry

In [ ]:
check_registry = CheckRegistry()
check_registry

In [ ]:
module_summary = TextExercise(
    exercise_description="""You can use this box to make general considerations, 
    or keep track of your doubts and questions about this notebook.""",
    exercise_registry=exercise_registry,
    exercise_title="Module comments",
    exercise_key="00"
)
display(module_summary)

# The rotation group 

Rotations describe the changes in the orientation in space of a rigid body relative to a fixed coordinate system. The mathematical description of rotations is notoriously tedious, with a plethora of different conventions that are often applied inconsistently in different works. 
If you are the kind of person who enjoys this stuff, you this [wikipedia article](https://en.wikipedia.org/wiki/Rotation_formalisms_in_three_dimensions) provides a comprehensive overview. 

In this exercise we are going to define rotations in terms of [Euler angles](https://en.wikipedia.org/wiki/Euler_angles) in the so-called ZYZ convention, in which the rotation is identified by three angles $(\alpha, \beta, \gamma)$ where $\alpha$ and $\gamma$ are periodic and can be chosen in the interval $[-\pi,\pi]$, and $\beta$ in the interval $[0,\pi]$.

To get a grasp of what Euler angles do, and why you need to define three angles to properly characterize the orientation of a structure, you can play around with the following visualization.

In [ ]:
ex01_pb =  ParameterPanel(
    alpha = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\alpha$'),
    beta = FloatSlider(value=0,min=0,max=np.pi,step=0.01,description=r'$\beta$'),
    gamma = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\gamma$'))

In [ ]:
ex01_fig = plt.figure(tight_layout=True)
ax01 = ex01_fig.add_subplot(111, projection='3d')
ex01_cuefig = CueFigure(ex01_fig) 

theta = np.linspace(0, 2 * np.pi, 20)
w = np.linspace(-0.5, 0.5, 10)
theta, w = np.meshgrid(theta, w)
R = 1
x = (R + w * np.cos(theta / 2)) * np.cos(theta)
y = (R + w * np.cos(theta / 2)) * np.sin(theta)
z = w * np.sin(theta / 2)

ex01_xyz =  np.array([x,y,z]).T

def update_01(code_exercise):    
    alpha, beta, gamma = code_exercise.parameters.values()
    cue_figure = code_exercise.cue_outputs[0]
    ax = cue_figure.figure.get_axes()[0]
    rot = rotation_matrix(alpha,beta,gamma)
    (x,y,z) = (ex01_xyz@rot.T).T
    ax.set_xlim([-2,2])
    ax.set_ylim([-2,2])
    ax.set_zlim([-2,2])
    dax = 2*np.eye(3)@rot.T    
    ax.quiver(0,0,0,*(dax[0]),color='r')
    ax.quiver(0,0,0,*(dax[1]),color='g')
    ax.quiver(0,0,0,*(dax[2]),color='b')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.plot_surface(x, y, z, color='gray')
    
    ax.set_aspect('auto')
    cue_figure.figure.subplots_adjust(left=0.0, right=1, top=1, bottom=0.0)

ce01 = CodeExercise(
            parameters=ex01_pb,
            cue_outputs = [ex01_cuefig],
            update_func = update_01,
            update_mode="continuous")

display(ce01)
ce01.run_update()

A group is a set of endowed with a composition operation. The composition operator is associative, and the set has to be closed under the action of the composition, have an identity element and for each element contain also its inverse. 

This means that it is possible to combine rotations, and that the composition of two rotations is itself a rotation $\hat{R}''=\hat{R}'\hat{R}$. Rotations are _not_ commutative, so the order of application of the rotation operators matters.

## Cartesian rotations

In practical terms, a rotation operator $\hat{R}$ parameterized by the Euler angles acts on a 3D object by applying the corresponding rotation matrix $\mathbf{R}$ to the Cartesian coordinates of all its points: if a structure $A$ has atomic positions $\mathbf{r}_i$ (each of these being a 3-vector corresponding to the Cartesian coordinates $(x,y,z)$) then the rotated structure $\hat{R}A$ has atomic coordinates $\mathbf{R}\mathbf{r}_i$. The same transformation is applied to all the properties $\mathbf{y}$ of $A$ that have a vectorial character, e.g. the dipole moment - so that the dipole of $\hat{R}A$ is $\mathbf{R}\mathbf{y}$. 

Note also that $SO(3)$ is an _orthogonal_ group, meaning that if $\mathbf{R}$ is the rotation matrix associated with $\hat{R}$, the matrix associated with $\hat{R}^{-1}$ is $\mathbf{R}^T$, and $\mathbf{RR}^T=\mathbf{R}^T\mathbf{R}$ is the identity. 

Let's consider a dataset that contains a few organic molecules, and for each of them the computed dipole noment $\boldsymbol{\mu}$ and polarizability $\boldsymbol{\alpha}$. These are structures from the "showcase" dataset from ([Yang et al. (2019)](http://doi.org/10.1038/s41597-019-0157-8)).

In [ ]:
frames_alphamu = ase.io.read('data/showcase.xyz', ":")

In [ ]:
dipoles_show = chemiscope.ase_vectors_to_arrows(frames_alphamu, "dipole_ccsd", scale=0.5)
dipoles_show["parameters"]["global"]["color"]="0xff8000"

alphas_show = chemiscope.ase_tensors_to_ellipsoids(frames_alphamu, "ccsd_pol", scale=0.2)
alphas_show["parameters"]["global"]["color"]="0xff0080"

In [ ]:
chemiscope.show(frames=frames_alphamu, 
                shapes={
                    "mu": dipoles_show,
                    "alpha": alphas_show
                       },
                mode="structure",
               settings=chemiscope.quick_settings(structure_settings={"shape":"mu"})
               )

In [ ]:
ex02_wci = CodeInput(
        function_name="rotate_atoms", 
        function_parameters="positions, dipole, rotm",
        docstring="""takes the positions and dipole of a structure and transforms them
        according to the given rotation matrix 
        
        :param positions: a (n_atoms,3) array containing the atomic positions
        :param dipole: a (3) array containing the dipole components
        :param rotm: a (3,3) array containing the rotation matrix
        
        :returns: (positions, dipole) - a tuple containing the transformed positions and dipole
""",
        function_body="""

# NB: be careful with how you can apply the rotations to the positions array

new_positions = positions.copy()
new_dipole = dipole.copy()

# Apply the rotation here. Be careful with the shape and layout of the arrays

return new_positions, new_dipole
"""
        )

In [ ]:
def update_02(code_exercise):
    output = code_exercise.cue_outputs[0]
    output.clear_output()
    rots = []
    f = frames_alphamu[0]
    for r in np.pi*np.array([[0,0,0],[0,0.125,0],[0,0.250,0],[0,0.375,0],[0,0.5,0],
                             [0.125,0.5,0],[0.25,0.5,0],[0.375,0.5,0],[0.5,0.5,0],
                             [0.5,0.5,0.125],[0.5,0.5,0.25],[0.5,0.5,0.375],[0.5,0.5,0.5]]):
        nf = deepcopy(f)
        nf.positions, nf.info["dipole_ccsd"] = ex02_wci.get_function_object()(
            f.positions, f.info["dipole_ccsd"], rotation_matrix(*r) )
        rots.append(nf)
    with output:
        dipoles_show = chemiscope.ase_vectors_to_arrows(rots, "dipole_ccsd", scale=0.5)
        dipoles_show["parameters"]["global"]["color"]="0xff8000"
        cs=chemiscope.show(rots, shapes={
                    "mu": dipoles_show,
                },
                mode="structure",
               settings=chemiscope.quick_settings(structure_settings={"shape":"mu", 
                                                                      "keepOrientation":True})
                          )
        cs.save("module_02-dipole_rotations.chemiscope.json.gz")
        display(cs)

ex02_reference_input = [{'positions':np.array([[0.,0,1],[1,2,0],[3,2,-1]]), 
                         'dipole':np.array([5.,6,7]),
                         'rotm': np.eye(3)},
                       {'positions':np.array([[0.,0,1],[1,2,0],[3,2,-1]]), 
                         'dipole':np.array([5.,6,7]),
                         'rotm': rotation_matrix(0,np.pi/2,0)}]
ex02_reference_output = [(np.array([[0.,0,1],[1,2,0],[3,2,-1]]),np.array([5.,6,7])),
                         (np.array([[ 1.00000000e+00,  0.00000000e+00,  2.22044605e-16],
         [ 2.22044605e-16,  2.00000000e+00, -1.00000000e+00],
         [-1.00000000e+00,  2.00000000e+00, -3.00000000e+00]]),
  np.array([ 7.,  6., -5.]))]
ex02_code_demo = CodeExercise(
    code= ex02_wci,
    check_registry=check_registry,
    cue_outputs = [CueObject()],
    update_func = update_02,
    exercise_key="02",
    exercise_registry=exercise_registry,
    exercise_title="Exercise 02: Moelcular rotations",
    exercise_description=mdwn("""
Implement a function that gets positions and dipole of a molecule and rotate them according to
the provided rotation matrix.
""")
)

check_registry.add_check(ex02_code_demo,
    asserts= [
        assert_type,
        assert_shape,
        assert_numpy_allclose,
    ],
     inputs_parameters=ex02_reference_input,
     outputs_references =ex02_reference_output)

In [ ]:
display(ex02_code_demo)

[Download chemiscope datafile](./module_02-dipole_rotations.chemiscope.json.gz)

There are however more complicated properties than those transforming as vectors. Go back to the dataset viewer, and change the visualizer settings to display the polarizability `alpha`. The polarizability describes the second order response of the energy of a molecule to an applied electric field, i.e.

$$
\alpha_{ab} = \frac{\partial^2 U}{\partial E_a \partial E_b}
$$

It is therefore a _tensor_ labeled by two Cartesian indices. In order to see how it transforms under rotations, you should consider that a rotation would affect the relation of the reference frame of the molecule to that of _both_ electric field vectors, so one needs to apply _two_ rotation matrices,

$$
\boldsymbol{\alpha}(\hat{R}A) = \mathbf{R}\boldsymbol{\alpha}(A)\mathbf{R}^T
$$

In [ ]:
ex03_wci = CodeInput(
        function_name="rotate_atoms_pol", 
        function_parameters="positions, alpha, rotm",
        docstring="""takes the positions and polarizability of a structure and transforms 
        them according to the given rotation matrix 
        
        :param positions: a (n_atoms,3) array containing the atomic positions
        :param alpha: a (3,3) matrix containing the polarizability
        :param rotm: a (3,3) array containing the rotation matrix
        
        :returns: (positions, alpha) - a tuple containing the transformed positions and polarizability
""",
        function_body="""

# NB: be careful with how you can apply the rotations to the positions array

new_positions = positions.copy()
new_alpha = alpha.copy()

# Apply the rotation here. Be careful with the shape and layout of the arrays

return new_positions, new_alpha
"""
        )

In [ ]:
def update_03(code_exercise):
    output = code_exercise.cue_outputs[0]
    output.clear_output()
    rots = []
    f = frames_alphamu[0]
    for r in np.pi*np.array([[0,0,0],[0,0.125,0],[0,0.250,0],[0,0.375,0],[0,0.5,0],
                             [0.125,0.5,0],[0.25,0.5,0],[0.375,0.5,0],[0.5,0.5,0],
                             [0.5,0.5,0.125],[0.5,0.5,0.25],[0.5,0.5,0.375],[0.5,0.5,0.5]]):
        nf = deepcopy(f)
        pol = nf.info["ccsd_pol"]
        pol = np.array([[pol[0], pol[3], pol[4]],[pol[3], pol[1], pol[5]],[pol[4], pol[5], pol[2]]])
        nf.positions, pol = ex03_wci.get_function_object()(
            nf.positions, pol, rotation_matrix(*r) )
        nf.info["ccsd_pol"][:] = [ pol[0,0], pol[1,1], pol[2,2], pol[0,1], pol[0,2], pol[1,2]] 
        rots.append(nf)
    with output:
        alphas_show = chemiscope.ase_tensors_to_ellipsoids(rots, "ccsd_pol", scale=0.2)
        alphas_show["parameters"]["global"]["color"]="0xff0080"
        cs=chemiscope.show(rots, shapes={
                    "alpha": alphas_show,
                },
                mode="structure",
               settings=chemiscope.quick_settings(structure_settings={"shape":"alpha", 
                                                                      "keepOrientation":True})
                          )
        cs.save("module_02-alpha_rotations.chemiscope.json.gz")
        display(cs)

ex03_reference_input = [{'positions':np.array([[0.,0,1],[1,2,0],[3,2,-1]]), 
                         'alpha':np.array([[5.,1,1],[1,3,0],[1,0,4]]),
                         'rotm': np.eye(3)},
                       {'positions':np.array([[0.,0,1],[1,2,0],[3,2,-1]]), 
                         'alpha':np.array([[8.,-1,1],[-1,9,0],[1,0,4]]),
                         'rotm': rotation_matrix(0,np.pi/2,0)}]
ex03_reference_output = [(np.array([[ 0.,  0.,  1.],
         [ 1.,  2.,  0.],
         [ 3.,  2., -1.]]),
  np.array([[5., 1., 1.],
         [1., 3., 0.],
         [1., 0., 4.]])),
 (np.array([[ 1.00000000e+00,  0.00000000e+00,  2.22044605e-16],
         [ 2.22044605e-16,  2.00000000e+00, -1.00000000e+00],
         [-1.00000000e+00,  2.00000000e+00, -3.00000000e+00]]),
  np.array([[ 4.00000000e+00, -2.22044605e-16, -1.00000000e+00],
         [-2.22044605e-16,  9.00000000e+00,  1.00000000e+00],
         [-1.00000000e+00,  1.00000000e+00,  8.00000000e+00]]))]

ex03_code_demo = CodeExercise(
    code= ex03_wci,
    check_registry=check_registry,
    cue_outputs = [CueObject()],
    update_func = update_03,
    exercise_key="03",
    exercise_registry=exercise_registry,
    exercise_title="Exercise 03: Tensor rotations",
    exercise_description=mdwn("""
Implement a function that gets positions and polarizability of a molecule and rotates 
them according to the provided rotation matrix.
""")
)

check_registry.add_check(ex03_code_demo,
    asserts= [
        assert_type,
        assert_shape,
        assert_numpy_allclose,
    ],
     inputs_parameters=ex03_reference_input,
     outputs_references =ex03_reference_output)

In [ ]:
display(ex03_code_demo)

[Download chemiscope datafile](./module_02-alpha_rotations.chemiscope.json.gz)

## Rotating tensors

The action of rotations on a Cartesian tensorial quantity can always be formulated as a matrix-vector multiplication, by "unrolling" the tensor and combining multiple rotation matrices together, e.g.

$$
\alpha_{(ab)} = \sum_{(a'b')} R_{(ab)(a'b')} \alpha_{(a'b')}
$$

where $R_{(ab)(a'b')}=R_{aa'}R_{bb'}$.  

In the visualization below you can see how the elements of the combined rotation matrix change with the Euler angles. 

In [ ]:
ex04_pb =  ParameterPanel(
    alpha = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\alpha$'),
    beta = FloatSlider(value=0,min=0,max=np.pi,step=0.01,description=r'$\beta$'),
    gamma = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\gamma$'))

ex04_fig = plt.figure(tight_layout=True)
ax04 = ex04_fig.add_subplot(111)
ex04_cuefig = CueFigure(ex04_fig) 

ex04_cbar = None
def update_04(code_exercise):
    global ex04_cbar
    alpha, beta, gamma = code_exercise.parameters.values()
    cue_figure = code_exercise.cue_outputs[0]
    ax = cue_figure.figure.get_axes()[0]
    rot = rotation_matrix(alpha,beta,gamma)
    ROT = np.einsum("ab,cd->acbd",rot, rot).reshape(9,9)
    
    fig = code_exercise.cue_outputs[0].figure
    ax = fig.get_axes()[0]

    cax=ax.matshow(ROT, cmap='seismic', vmin=-1, vmax=1)
    if ex04_cbar is None:
        ex04_cbar = fig.colorbar(cax, ax=ax, orientation='vertical' )
    else:
        ex04_cbar.update_normal(cax)

    ax.set_xlabel("(ab)")
    ax.set_ylabel("(a'b')")    

In [ ]:
ex04_code_demo = CodeExercise(
            parameters= ex04_pb,            
            cue_outputs = [ex04_cuefig],
            update_func = update_04,
    update_mode="continuous",
    #exercise_key="04",
    #exercise_registry=exercise_registry,
    exercise_title="Exercise 04",
    exercise_description=mdwn("")
)

display(ex04_code_demo)
ex04_code_demo.run_update()

In [ ]:
ex04_txt = TextExercise(
    exercise_description="""
Play around with the parameters. Does the matrix have any obvious structure? 
How many multiplications would you have to perform to rotate the polarizability 
using the tensorial (two rotations) notation? And how about the combined (single matrix)
case?""",
    exercise_registry=exercise_registry,
    exercise_key="04",
    exercise_title=""
)
display(ex04_txt)

In [ ]:
ex04b_txt = TextExercise(
    exercise_description=mdwn(r"""
Given we have seen there are different ways to transform a tensor such as the 
polarizability, we can wonder if there are _better_ ways to apply a rotation to a tensor.
For example, consider the _trace_ of the polarizability, $\alpha_{xx}+\alpha_{yy}+\alpha_{zz}$.
How does it transform under rotations?"""),
    exercise_registry=exercise_registry,
    exercise_key="04b",
    exercise_title="Polarizability trace"
)
display(ex04b_txt)

## Rotations and spherical harmonics

[Spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics) are special functions of the polar angles $(\theta, \phi)$ that can be obtained as the orthogonal solutions of the Laplacian eigenvalue problem on the sphere $\nabla^2 Y^m_l(\theta, \phi) = \epsilon_l Y^m_l(\theta, \phi)$. The spherical harmonics are indexed by two integers, $l\ge 0$ and $-l\le m \le l$. 
Much as for rotations, there are many different convention used when defining spherical harmonics. We use real valued spherical harmonics, which are the same used in the construction of the density expansion features. 

In [ ]:
# Define the resolution of the sphere
num_points = 40

# Create the angles
theta = np.linspace(0, np.pi, num_points)
phi =   np.linspace(0, 2*np.pi, 2*num_points)
theta, phi = np.meshgrid(theta, phi)

# Convert to Cartesian coordinates
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

ex05_xyz =  np.array([x,y,z])

# Define a function on the sphere
# Example function: cos(theta) + sin(phi)
ex05_sph = sph.compute(ex05_xyz.T.reshape(-1,3)).reshape(theta.shape[1], theta.shape[0], -1).T

In [ ]:
ex05_mslider = IntSlider(value=0,min=0,max=8,step=1,description=r'$m$')
ex05_pb =  ParameterPanel(
    l = IntSlider(value=1,min=0,max=8,step=1,description=r'$l$'),
    m = ex05_mslider
)

ex05_fig = plt.figure(tight_layout=True)
ax05 = ex05_fig.add_subplot(111, projection='3d')
ex05_cuefig = CueFigure(ex05_fig) 

def update_05(code_exercise):    
    l, m = code_exercise.parameters.values()
    # updates the range of the m slider
    ex05_mslider.min = -l 
    ex05_mslider.max = l
    if m>l: 
        m=l
    if m<-l:
        m=-l
    cue_figure = code_exercise.cue_outputs[0]
    ax = cue_figure.figure.get_axes()[0]
    ax.set_xlim([-1.0,1.0])
    ax.set_ylim([-1.0,1.0])
    ax.set_zlim([-1.0,1.0])
    
    color_map = lambda x:  mpl.colormaps['seismic']((x-x.min())/(1e-15+x.max()-x.min()))
    x,y,z = ex05_xyz
    lm=l*l+l+m
    # Plot the sphere with colors
    ax.plot_surface(x, y, z, rstride=1, antialiased=True,
                    cstride=1, shade=True, facecolors=color_map(ex05_sph[lm]) )
    ax.set_axis_off()
    ax.set_aspect('auto')
    cue_figure.figure.subplots_adjust(left=0.0, right=1, top=1, bottom=0.0)
    
ce05 = CodeExercise(
            parameters=ex05_pb,
            cue_outputs = [ex05_cuefig],
            update_func = update_05,
            update_mode="continuous")

Use the viewer below to visualize different spherical harmonics. Spherical harmonics are connected to the solution of the Schrödinger equation in a central potential, and more broadly to the definition of angular momentum in quantum mechanics. 

In [ ]:
display(ce05)
ce05.run_update()

Spherical harmonics are also special because of their connection with the properties of the rotation group. In fact, it is possible to define special matrix-valued functions (the [Wigner D matrices](https://en.wikipedia.org/wiki/Wigner_D-matrix#Relation_to_spherical_harmonics_and_Legendre_polynomials) $D^l_{mm'}(\hat{R})$) that describe how spherical harmonics transform under rotations. 

Essentially, all the spherical harmonics with the same $l$ should be treated as a vector, and then 

$$
Y^m_l(\hat{R}\theta, \hat{R}\phi) = \sum_{m'}D^l_{mm'}(\hat{R}) Y^{m'}_l(\theta,\phi)
$$

We are going to see how this can be realized by writing a function that takes a list of 3-vectors that are points on the surface of a sphere, and use a utility function (a wrapper to the `sphericart` library) to compute the spherical harmonics for the initial and rotated points, and apply the rotation directly to the spherical harmonics using the Wigner matrix for the rotation.

In [ ]:
def spherical_harmonics(l, r):
    return sph.compute(r)[:,l**2:(l+1)**2]
ex06_wci = CodeInput(
        function_name="rotate_ylm", 
        function_parameters="xyz, rotm, wigd, spherical_harmonics",
        docstring="""computes rotated spherical harmonics in two different ways, 
        first by rotating the input positions, and then by rotating the spherical 
        harmonics computed at the initial positions. 
        
        :param xyz: a (n,3) array containing the positions at which to 
             compute the spherical harmonics
        :param rotm: a (3,3) array containing the rotation matrix
        :param wigd: a (2l+1,2l+1) array containing the wigner D matrix
        :param spherical_harmonics: a function of (l, xyz) that computes the spherical
             harmonics, returning a (n, 2l+1) array
        :returns: (rylm1, rylm2) - a tuple containing the spherical harmonics computed
             using Ylm(R xyz) and Dlm Ylm(R)
""",
        function_body="""

l = (wigd.shape[0]-1)//2   # guesses the order of Ylm from the shape of Dl
ylm = spherical_harmonics(l, xyz)

rxyz = xyz   # <--- apply rotation here
rylm_1 = spherical_harmonics(l, rxyz)

rylm_2 = ylm  # <--- apply rotation here

return rylm_1, rylm_2
"""
)

In [ ]:
ex06_mslider = IntSlider(value=0,min=0,max=8,step=1,description=r'$m$')
ex06_pb =  ParameterPanel(
    l = IntSlider(value=1,min=0,max=8,step=1,description=r'$l$'),
    m = ex06_mslider,
    alpha = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\alpha$'),
    beta = FloatSlider(value=0,min=0,max=np.pi,step=0.01,description=r'$\beta$'),
    gamma = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\gamma$'),
)

ex06_fig = plt.figure(tight_layout=True)
ax061 = ex06_fig.add_subplot(121, projection='3d')
ax062 = ex06_fig.add_subplot(122, projection='3d')
ex06_cuefig = CueFigure(ex06_fig) 
for ax in [ax061, ax062]:
    ax.mouse_init(rotate_btn=None, pan_btn=None, zoom_btn=None)
    ax.set_xlim([-1.0,1.0])
    ax.set_ylim([-1.0,1.0])
    ax.set_zlim([-1.0,1.0])
    ax.set_axis_off()

ex06_xyz = ex05_xyz

def update_06(code_exercise):    
    l, m, alpha, beta, gamma = code_exercise.parameters.values()
    # updates the range of the m slider
    ex06_mslider.min = -l 
    ex06_mslider.max = l
    if m>l: 
        m=l
    if m<-l:
        m=-l
    cue_figure = code_exercise.cue_outputs[0]
    ax1, ax2 = cue_figure.figure.get_axes()[:2]
        
    color_map = lambda x:  mpl.colormaps['seismic']((x-x.min())/(1e-15+x.max()-x.min()))
    shape = ex06_xyz.shape 
    x, y, z = ex06_xyz
    xyz = ex06_xyz.reshape(3, -1).T
    rotm = rotation_matrix(alpha, beta, gamma)
    wigd = wigner_d_real(l, alpha, beta, gamma)
    
    rlm1, rlm2 = ex06_wci.get_function_object()(xyz, rotm, wigd, spherical_harmonics)
    
    ax1.plot_surface(x, y, z, rstride=1, antialiased=True,
                    cstride=1, shade=True, facecolors=color_map(rlm1[:,l+m].reshape(shape[1:]) ) )
    ax2.plot_surface(x, y, z, rstride=1, antialiased=True,
                    cstride=1, shade=True, facecolors=color_map(rlm2[:,l+m].reshape(shape[1:]) ) )
    for ax in [ax1, ax2]:
        ax.set_xlim([-1.0,1.0])
        ax.set_ylim([-1.0,1.0])
        ax.set_zlim([-1.0,1.0])
        ax.set_axis_off()
        
ce06 = CodeExercise(
            code=ex06_wci,
            parameters=ex06_pb,
            cue_outputs = [ex06_cuefig],
            update_func = update_06,
            update_mode="manual",
    exercise_key="06",
    exercise_registry=exercise_registry,
    exercise_title="Exercise 06: Two ways of rotating spherical harmonics",
    exercise_description=mdwn(r"""Write a function that computes spherical harmonics
    in a rotated reference frame, by rotating the position at which the $Y^m_l$ are 
    evaluated, and by rotating the set of spherical harmonics computed at the initial 
    location. """)
)

In [ ]:
display(ce06)

## Irreducible representations

Wigner $D$ matrices have a very important property, that makes them central in the theory and practice of $SO(3)$: given a set of objects that transform under rotations (e.g. the entries in the polarizability tensor) it is possible to determine linear combinations of those entries that transform under rotations as with the application of a Wigner matrix. 

What is more, these special linear combinations are the _smallest_ possible sets of quantities that are mixed by a rotation, and it is not possible to simplify them any further. We have already encountered an example of such linear transformation: the trace of the polarizability is (proportional to) the linear combination that is constant (i.e. that is transformed by $D^0_{00}=1$). More generally any product of Cartesian components can be transformed into blocks that transform as spherical harmonics.

Performing these transformations is rather tedious (as they depend on the convention used to define the Wigner matrices) so you'll have to trust the implementation provided with this exercise. `cart2sph` takes a Cartesian tensor and transforms it into a 9-vector in which the first element transforms as a $l=0$ spherical harmonics, the elements `[1:4]` as $l=1$, and `[4:9]` as $l=2$. 

```python
print(alpha)
print(cart2sph(alpha))
```

In [ ]:
alpha = six2cart(frames_alphamu[0].info["ccsd_pol"])
print(alpha)
print(cart2sph(alpha))

Note that the $l=1$ terms are zero; this is because the polarizability is a _symmetric_ tensor, and has only six independent entries: the asymmetric part of the tensor would be transformed into an $l=1$ block.

Rotating a vector in this form can be achieved by a _block diagonal_ matrix with Wigner $D$ matrices on the diagonal. Obviously the whole point of expressing a tensor in its irreducible form is to manipulate separately the different blocks, but the following visualization displays the block-diagonal form, to be compared with the $9\times 9 $ transformation in Exercise 04. 

In [ ]:
ex07_pb =  ParameterPanel(
    alpha = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\alpha$'),
    beta = FloatSlider(value=0,min=0,max=np.pi,step=0.01,description=r'$\beta$'),
    gamma = FloatSlider(value=0,min=-np.pi,max=np.pi,step=0.01,description=r'$\gamma$'))

ex07_fig = plt.figure(tight_layout=True)
ax07 = ex07_fig.add_subplot(111)
ex07_cuefig = CueFigure(ex07_fig) 

ex07_cbar = None
def update_07(code_exercise):
    global ex07_cbar
    alpha, beta, gamma = code_exercise.parameters.values()
    cue_figure = code_exercise.cue_outputs[0]
    ax = cue_figure.figure.get_axes()[0]

    ROT = scipy.linalg.block_diag(
        wigner_d_real(0, alpha, beta, gamma), 
        wigner_d_real(1, alpha, beta, gamma), 
        wigner_d_real(2, alpha, beta, gamma)
    )
    
    fig = code_exercise.cue_outputs[0].figure
    ax = fig.get_axes()[0]

    cax=ax.matshow(ROT, cmap='seismic', vmin=-1, vmax=1)
    if ex07_cbar is None:
        ex07_cbar = fig.colorbar(cax, ax=ax, orientation='vertical' )
    else:
        ex07_cbar.update_normal(cax)

    ax.set_xlabel("(lm)")
    ax.set_ylabel("(lm)")    

In [ ]:
ex07_code_demo = CodeExercise(
            parameters= ex07_pb,            
            cue_outputs = [ex07_cuefig],
            update_func = update_07,
    update_mode="continuous",
    #exercise_key="04",
    #exercise_registry=exercise_registry,
    exercise_title="Exercise 07",
    exercise_description=mdwn("")
)

display(ex07_code_demo)
ex07_code_demo.run_update()

In [ ]:
ex07_txt = TextExercise(
    exercise_description=mdwn(r"""
How many multiplications would you have to perform to apply the rotation exploiting 
the block structure? And how about if you also exploited knowledge that the $l=1$ 
block is zero?"""),
    exercise_registry=exercise_registry,
    exercise_key="07",
    exercise_title=""
)
display(ex07_txt)

# Equivariance: a primer

_Equivariance_ indicates the property of a function for which the inputs and outputs are subject to the action of the same symmetries, and which commutes with the application of the symmetries, that is: $f(\hat{S}A) = \hat{S} f(A)$. _Invariance_ can be seen as a special case, in which  $f(\hat{S}A) = f(A)$.
This section focuses in particular on the case of 3D rotations and inversion - in technical terms the $O(3)$ group symmetries - and their combination with translations - the three-dimensional Euclidean group $E(3)$. 

In [ ]:
from IPython.display import display, Javascript

script = """
document.querySelectorAll('.jp-Cell-inputWrapper').forEach(function(element) {
    element.style.display = ''; // 'none' to hide
});
"""
display(Javascript(script))

In [ ]:
# set CSS style for code-hide
scwidgets.get_css_style()

In [ ]:
exercise_registry = ExerciseRegistry(filename_prefix="module_02")
exercise_registry

In [ ]:
check_registry = CheckRegistry()
check_registry

In [ ]:
show_script = """
document.querySelectorAll('.jp-Cell-inputWrapper').forEach(function(element) {
    element.style.display = '';
});
"""
display(Javascript(show_script))

In [ ]:
module_summary = TextExercise(
    exercise_description="""You can use this box to make general considerations, 
    or keep track of your doubts and questions about this notebook.""",
    exercise_registry=exercise_registry,
    exercise_title="Module comments",
    exercise_key="00"
)
display(module_summary)

Let's consider this dataset, which contains a collection of configurations for a single water molecule. The configurations are generated by distorting an equilibrium configuration along the bending mode, and the asymmetric stretching coordinate. Each frame contains also the energy and dipole moment, computed with the Partridge-Schwenke monomer potential ([Partridge, Schwenke, J. Chem. Phys. (1997)](http://doi.org/10.1063/1.473987)). 

In [ ]:
h2o_frames = ase.io.read("data/water_energy-dipole.xyz", ":")

h2o_energy = np.zeros(len(h2o_frames))
h2o_dipole = np.zeros((len(h2o_frames),3))
h2o_force = np.zeros((len(h2o_frames),3,3))
for fi, f in enumerate(h2o_frames):
    h2o_energy[fi] = f.info['energy']
    h2o_dipole[fi] = f.info['dipole']
    h2o_force[fi] = f.arrays['force']

<a id="data-driven"> </a>

In [ ]:
ex07_figure, ex07_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex07_output = CueFigure(ex07_figure)

ex07_xy = frame_iron.positions[selection,:2]
ex07_cbar = None
def update_07(code_exercise):
    global ex07_cbar
    rcut, delta, rs = code_exercise.parameters.values()
    fig = code_exercise.cue_outputs[0].figure
        
    ax = fig.get_axes()[0]
    ax.axis('off')
    vals = np.zeros(len(selection))
    acsf=ex03_wci.get_function_object()
    for i in tqdm(range(len(vals))):
        vals[i] = acsf(neigh_dr[neigh_i[i]:neigh_i[i]+neigh_sz[i]], delta=delta, rs=rs, rcut=rcut)
    cax=ax.scatter(ex07_xy[:,0], ex07_xy[:,1], c=vals, marker='.', s=5,
                  vmin=vals.min(), vmax = vals.max() )       
    
    if ex07_cbar is None:
        ex07_cbar = fig.colorbar(cax, ax=ax )
    else:
        ex07_cbar.update_normal(cax)

    ax = fig.get_axes()[1]
    ax.hist(vals, color='red', bins=50)
    ax.set_xlabel(r"$\xi$")
    ax.set_ylabel(r"counts")

ex07_pb =  ParameterPanel(
    rcut = FloatSlider(value=5,min=3,max=8,step=0.1,description=r'$r_{cut}$ / Å'),
    delta = FloatSlider(value=0.5,min=0.1,max=2,step=0.1,description=r'$\Delta$ / Å'),
    rs = FloatSlider(value=5,min=3,max=8,step=0.1,description=r'$r_s$ / Å'),
    )

ex07_code_demo = CodeExercise(
    parameters= ex07_pb,
    cue_outputs = [ex07_output],
    update_func = update_07,
    update_mode="manual",
    #exercise_key="07",
    exercise_title="Exercise 07: Radial ACSF as a structural fingerprint",
    exercise_description=mdwn("""
Play around with the ACSF parameters (this exercise uses the function *you* implemented
in exercise 03), and see if you can find values that differentiate clearly between liquid-like
and solid-like environments.
"""
))

In [ ]:
display(ex07_code_demo)
ex07_code_demo.run_update()

In [ ]:
ex07b_txt = TextExercise(
    exercise_description="""
Observe the variability in the descriptor values.
Why do you think it's hard to find good values for the ACSF to make it good at discriminating?
Think also at the radial function plots in exercise 02.
    """,
    exercise_registry=exercise_registry,
    exercise_key="07b",
    exercise_title=""
)
display(ex07b_txt)